# Test_3D_conv

In [3]:
# importing the libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# for reading and displaying images
# from skimage.io import imread
import matplotlib.pyplot as plt

# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score


# PyTorch libraries and modules
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import h5py
# from plot3D import *

def array_to_color(array, cmap="Oranges"):
    s_m = plt.cm.ScalarMappable(cmap=cmap)
    return s_m.to_rgba(array)[:,:-1]


def rgb_data_transform(data):
    data_t = []
    for i in range(data.shape[0]):
        data_t.append(array_to_color(data[i]).reshape(16, 16, 16, 3))
    return np.asarray(data_t, dtype=np.float32)
  

In [4]:

with h5py.File("/home/miemie/Dropbox/PhD/project_MD_ML/full_dataset_vectors.h5", "r") as hf:
#   print(dir(hf))  # H5 is like a dictionary 
#   print(hf.keys())
  # Data is stored as 1D array. 
  # Should reshape after loading 
  X_train = hf["X_train"][:]
  targets_train = hf["y_train"][:]
  X_test = hf["X_test"][:] 
  targets_test = hf["y_test"][:]  
  
  
  


# Determine sample shape
sample_shape = (16, 16, 16, 3)

# Reshape data into 3D format
X_train = rgb_data_transform(X_train)
print(X_train.shape)
X_test = rgb_data_transform(X_test)


train_x = torch.from_numpy(X_train).float()
train_y = torch.from_numpy(targets_train).long()
test_x = torch.from_numpy(X_test).float()
test_y = torch.from_numpy(targets_test).long()
# The data set is shaped (10000, 16, 16, 16, 3)



(10000, 16, 16, 16, 3)


In [25]:
16**3*3

12288

In [6]:
batch_size = 100 #We pick beforehand a batch_size that we will use for the training

# Pytorch train and test sets
# torch.utils.data.TensorDataset is like the zip to bind train_x and train_y
train = torch.utils.data.TensorDataset(train_x, train_y)
test = torch.utils.data.TensorDataset(test_x, test_y)
# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)


In [15]:
num_classes = 10

# Create CNN Model
class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel, self).__init__()
    self.conv_layer1 = self._conv_layer_set(3, 32)
    self.conv_layer2 = self._conv_layer_set(32, 64)
    self.fc1 = nn.Linear(2**3*64, 128)
    self.fc2 = nn.Linear(128, num_classes)
    self.relu = nn.LeakyReLU()
    self.batch = nn.BatchNorm1d(128)
    self.drop = nn.Dropout(p=0.15)
    

  def _conv_layer_set(self, in_c, out_c):
    conv_layer = nn.Sequential(
      nn.Conv3d(in_c, out_c, kernel_size=(3, 3, 3), padding=0),
      nn.LeakyReLU(),
      nn.MaxPool3d((2, 2, 2)),
    )
    return conv_layer


  def forward(self, x):
    # Set 1
    out = self.conv_layer1(x)
    print(out.shape)
    out = self.conv_layer2(out)
    print(out.shape)
    out = out.view(out.size(0), -1)
    print(out.shape)
    out = self.fc1(out)
    print(out.shape)
    out = self.relu(out)
    print(out.shape)
    out = self.batch(out)
    print(out.shape)
    out = self.drop(out)
    print(out.shape)
    out = self.fc2(out)
    print(out.shape)
    return out
      
#Definition of hyperparameters
n_iters = 4500



num_epochs = n_iters / (len(train_x) / batch_size)
num_epochs = int(num_epochs)

# Create CNN
model = CNNModel()
#model.cuda()
print(model)

# Cross Entropy Loss 
error = nn.CrossEntropyLoss()

# SGD Optimizer
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

CNNModel(
  (conv_layer1): Sequential(
    (0): Conv3d(3, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer2): Sequential(
    (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (batch): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.15, inplace=False)
)


In [16]:
count = 0
loss_list = []
iteration_list = []
accuracy_list = []
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    print(type(images))
    train = Variable(images.view(100,3,16,16,16))
    labels = Variable(labels)
    # Clear gradients
    optimizer.zero_grad()
    # Forward propagation
    outputs = model(train)
    # Calculate softmax and ross entropy loss
    loss = error(outputs, labels)
    # Calculating gradients
    loss.backward()
    # Update parameters
    optimizer.step()

    count += 1
    if count % 50 == 0:
      # Calculate Accuracy         
      correct = 0
      total = 0
      # Iterate through test dataset
      for images, labels in test_loader:
        test = Variable(images.view(100,3,16,16,16))
        # Forward propagation
        outputs = model(test)
        # Get predictions from the maximum value
        predicted = torch.max(outputs.data, 1)[1]
        # Total number of labels
        total += len(labels)
        correct += (predicted == labels).sum()

      accuracy = 100 * correct / float(total)

      # store loss and iteration
      loss_list.append(loss.data)
      iteration_list.append(count)
      accuracy_list.append(accuracy)
    if count % 500 == 0:
      # Print Loss
      print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data, accuracy))

<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torc

torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([

<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torc

<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torc

<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torc

...

<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torc

torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torc

torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch

<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([100, 10])
<class 'torch.Tensor'>
torch.Size([100, 32, 7, 7, 7])
torch.Size([100, 64, 2, 2, 2])
torch.Size([100, 512])
torc

KeyboardInterrupt: 